<font color = 'red' size = 6>Load a pre trained model</font>

In [ ]:
import tensorflow as tf

# Load the pre-trained model
model = tf.saved_model.load('ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model')

<font size = 6 color = 'orange'>Load and preprocess the image</font>

In [5]:
import cv2
import numpy as np

# Load the image
image_path = 'image.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Convert image to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Resize the image to the size the model expects
input_tensor = tf.convert_to_tensor(image_rgb)
input_tensor = input_tensor[tf.newaxis]

# Run the model
detections = model(input_tensor)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# The model returns a dictionary of tensors
detection_boxes = detections['detection_boxes'][0].numpy()
detection_classes = detections['detection_classes'][0].numpy().astype(np.int32)
detection_scores = detections['detection_scores'][0].numpy()

# Define a threshold for detection confidence
confidence_threshold = 0.5
height, width, _ = image.shape

# Draw the bounding boxes and labels on the image
for i in range(len(detection_boxes)):
    if detection_scores[i] >= confidence_threshold:
        box = detection_boxes[i]
        class_id = detection_classes[i]
        score = detection_scores[i]
        
        # Convert to pixel coordinates
        y_min, x_min, y_max, x_max = box
        x_min = int(x_min * width)
        x_max = int(x_max * width)
        y_min = int(y_min * height)
        y_max = int(y_max * height)
        
        # Draw the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        # Draw the label
        label = f'Class {class_id}: {score:.2f}'
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [ ]:
# Convert back to BGR for OpenCV
image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# Display the image
cv2.imshow('Object Detection', image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
